In [12]:
import os
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dropout, Dense, GlobalAveragePooling2D,BatchNormalization,Flatten
from tensorflow.keras.models import Model,Sequential
import efficientnet.keras as efn
from tensorflow.keras.applications import EfficientNetB0
from PIL import Image
from sklearn.model_selection import train_test_split
import os
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [13]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)


In [14]:
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')


In [15]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2186279720003028340
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5933760512
locality {
  bus_id: 1
  links {
  }
}
incarnation: 12593986780896218291
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3060 Ti, pci bus id: 0000:2b:00.0, compute capability: 8.6"
xla_global_id: 416903419
]


2023-06-26 13:22:41.655763: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-26 13:22:41.655963: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-26 13:22:41.656087: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [16]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [17]:
# Set GPU device
os.environ['CUDA_VISIBLE_DEVICES'] = '0'  # Replace '0' with the GPU device ID

In [18]:
# Set seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

In [19]:
# Define image dimensions and batch size
img_height, img_width = 224,224
batch_size = 8

In [20]:
train_dir = "/media/maged/New Volume/NU/1-Spring 23/Grad Project 2/datasets again/Defacto + others/train"
test_dir = "/media/maged/New Volume/NU/1-Spring 23/Grad Project 2/datasets again/Defacto + others/test"
validation_dir = "/media/maged/New Volume/NU/1-Spring 23/Grad Project 2/datasets again/Defacto + others/validation"

In [21]:
# Create data generators with augmentation for training and validation sets
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    shear_range=0.2  

)


In [22]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='training',
    shuffle=True
)

Found 20800 images belonging to 2 classes.


In [23]:
train_generator.class_indices

{'authentic': 0, 'tampered': 1}

In [24]:
# Create data generator without augmentation for test set
test_datagen = ImageDataGenerator(rescale=1./255)

In [25]:
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True
)

Found 7311 images belonging to 2 classes.


In [26]:
# Rescaling for the validation set
validation_datagen = ImageDataGenerator(rescale=1.0 / 255)

In [27]:
# Flow validation images from the validation directory
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'
)

Found 8443 images belonging to 2 classes.


# Create the base EfficientNetB0 model
efnb0 = EfficientNetB0(weights= None, include_top=False, input_shape=(img_height, img_width, 3))

# Freeze the base model
efnb0.trainable = False

# Create the inputs tensor
inputs = tf.keras.Input(shape=(img_height, img_width, 3))

# Pass the inputs through EfficientNetB0
x = efnb0(inputs, training=False)

# Add global average pooling layer
x = GlobalAveragePooling2D()(x)

# Add dropout layer
x = Dropout(0.5)(x)

# Add output dense layer
outputs = Dense(1, activation='sigmoid')(x)

# Create the model
model = Model(inputs=inputs, outputs=outputs)

In [28]:
base_model = EfficientNetB0(include_top=False, weights= None, input_shape=(224, 224, 3))
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x) 
model = Model(inputs=base_model.input, outputs=predictions)


2023-06-26 13:22:43.378792: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-26 13:22:43.378948: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-26 13:22:43.379163: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [29]:
# Compile the model
model.compile(optimizer= 'Adam', loss='binary_crossentropy', metrics=['accuracy'])

In [30]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 224, 224, 3)  0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 224, 224, 3)  7           ['rescaling[0][0]']              
                                                                                                  
 stem_conv_pad (ZeroPadding2D)  (None, 225, 225, 3)  0           ['normalization[0][0]']      

checkpoint_path = '/home/maged/models/model_{epoch:02d}_accuracy_{accuracy:.4f}.h5'

# Define the ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint(
    checkpoint_path,
    monitor='accuracy',
    verbose=1,
    save_best_only=False,
    save_weights_only=False,
    mode='auto',
    save_freq='epoch'
)

checkpoint_path = '/home/maged/Final_model_EffNetB4.9.h5'

model.load_weights(checkpoint_path)

In [31]:
# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.000001, verbose=1)

In [32]:
class_weights_dict = {0: 1, 1: 3}  # Assuming class 0 is authentic and class 1 is tampered



In [33]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    callbacks=[early_stopping, reduce_lr]
    #,class_weight=class_weights_dict
)

Epoch 1/50


2023-06-26 13:22:45.737130: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-06-26 13:23:02.732064: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600


   5/2600 [..............................] - ETA: 11:33 - loss: 0.8421 - accuracy: 0.3750

2023-06-26 13:23:13.457796: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x343595e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-06-26 13:23:13.457816: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Ti, Compute Capability 8.6
2023-06-26 13:23:13.686978: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-06-26 13:23:15.057938: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2600/2600 [==============================] - ETA: 0s - loss: 0.7194 - accuracy: 0.5133

2023-06-26 13:35:45.650962: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


2600/2600 [==============================] - 1011s 379ms/step - loss: 0.7194 - accuracy: 0.5133 - val_loss: 0.6951 - val_accuracy: 0.4744 - lr: 0.0010
Epoch 2/50
2600/2600 [==============================] - 622s 239ms/step - loss: 0.7023 - accuracy: 0.5149 - val_loss: 0.6929 - val_accuracy: 0.4800 - lr: 0.0010
Epoch 3/50
2600/2600 [==============================] - 317s 122ms/step - loss: 0.6928 - accuracy: 0.5136 - val_loss: 0.6939 - val_accuracy: 0.4786 - lr: 0.0010
Epoch 4/50
2600/2600 [==============================] - 293s 113ms/step - loss: 0.6927 - accuracy: 0.5136 - val_loss: 0.6912 - val_accuracy: 0.4810 - lr: 0.0010
Epoch 5/50
2600/2600 [==============================] - 290s 112ms/step - loss: 0.7854 - accuracy: 0.5144 - val_loss: 0.6963 - val_accuracy: 0.4709 - lr: 0.0010
Epoch 6/50
2600/2600 [==============================] - ETA: 0s - loss: 0.6929 - accuracy: 0.5135
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
2600/2600 [===================

In [34]:
model.save("EffNetB0_copymove.h5")

In [35]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test loss: {test_loss:.4f}')
print(f'Test accuracy: {test_accuracy:.4f}')

  1/914 [..............................] - ETA: 4:05 - loss: 0.7002 - accuracy: 0.3750

2023-06-26 14:24:24.466973: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


914/914 [==============================] - 200s 219ms/step - loss: 0.6934 - accuracy: 0.5023
Test loss: 0.6934
Test accuracy: 0.5023


In [36]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

In [37]:

# Plotting the test loss and test accuracy
plt.figure(fdigsize=(8, 4))
plt.bar(['Test Loss', 'Test Accuracy'], [test_loss, test_accuracy])
plt.ylim([0, 1])
plt.ylabel('Value')
plt.title('Model Evaluation on Test Set')
plt.show()

TypeError: __init__() got an unexpected keyword argument 'fdigsize'

In [ ]:
um_test_samples = len(test_generator.labels)

# Generate predictions for the test data using the model
y_pred = model.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=1)

# Get the true labels from the test data generator
y_true = test_generator.labels

# Generate the confusion matrix
cm = confusion_matrix(y_true, y_pred_classes)

# Plot the confusion matrix
class_labels = test_generator.class_indices.keys()
plt.figure(figsize=(6, 6))
sns.heatmap(cm, annot=True, cmap='Blues', fmt='d',
            xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
from sklearn import metrics

actual = y_true
predicted = y_pred_classes

confusion_matrix = metrics.confusion_matrix(actual, predicted)

cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [False, True])

cm_display.plot()
plt.show()

In [ ]:
plt.figure(figsize=(10, 4))
plt.subplot(1, 2# Set global dtype policy to mixed_float16
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy), 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'])

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'])

plt.tight_layout()
plt.show()